In [5]:
import sys
import time
import struct
import socket
import requests
import dns.resolver

# Function to check if a string is a valid IP address
def is_valid_ip(address):
    try:
        socket.inet_pton(socket.AF_INET, address)
        return True
    except socket.error:
        return False

# Function to resolve a domain to its IP address
def get_ip(domain, dns_server='8.8.8.8'):
    if domain.startswith("https://"):
        domain = domain[8:]  # Remove "https://" prefix
    if domain.startswith("http://"):
        domain = domain[8:]  # Remove "http://" prefix
    resolver = dns.resolver.Resolver()
    resolver.nameservers = [dns_server]
    answers = resolver.resolve(domain, 'A')
    for rdata in answers:
        return rdata.address
    
# Function to get the domain name from an IP address
def get_domain_name(ip_address):
    try:
        domain_name = socket.gethostbyaddr(ip_address)[0]
        return domain_name
    except socket.herror:
        return "No domain name found"

# Function to get the IP address from a domain name
def get_ip_address(domain_name):
    try:
        ip_address = socket.gethostbyname(domain_name)
        return ip_address
    except socket.gaierror:
        return "Unable to resolve the domain"

# Function to send a ping request to a given IP address
def send_ping(dest_ip):
    icmp_socket = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_ICMP)
    icmp_packet = struct.pack('!BBHHH', 8, 0, 0, 0, 1) + b'pingdata'
    checksum = 0
    for i in range(0, len(icmp_packet), 2):
        checksum += (icmp_packet[i] << 8) + icmp_packet[i + 1]
    checksum = (checksum >> 16) + (checksum & 0xFFFF)
    checksum = ~checksum & 0xFFFF
    icmp_packet = struct.pack('!BBHHH', 8, 0, checksum, 0, 1) + b'pingdata'
    icmp_socket.sendto(icmp_packet, (dest_ip, 1))
    start_time = time.time()
    try:
        response, _ = icmp_socket.recvfrom(1024)
        icmp_type = struct.unpack('!B', response[20:21])[0]
        if icmp_type == 0:
            elapsed_time = time.time() - start_time
            return elapsed_time
    except socket.error:
        pass
    return None

# Main function to execute the ping command
def main(dest, num_requests):
    if dest.startswith("https://"):
        dest = dest[8:] # Remove "https://" prefix
    elif dest.startswith("http://"):
        dest = dest[7:] # Remove "http://" prefix

    if is_valid_ip(dest):
        dest_ip = dest
        try:
            domain = socket.gethostbyaddr(dest_ip)[0]
            print(f'The domain name for {dest_ip} is {domain}')
        except socket.herror:
            print("No domain name found for this IP address.")
        domain = None
    else:
        domain = dest
        try:
            dest_ip = socket.gethostbyname(domain)
            print(f'The IP address for {domain} is {dest_ip}')
        except socket.gaierror:
            print("Unable to resolve the domain.")
            return

    total_time = 0
    successful_responses = 0
    for i in range(num_requests):
        response_time = send_ping(dest_ip)
        if response_time is not None:
            total_time += response_time
            successful_responses += 1
            print(f'Request {i+1} - Ping: {response_time} ms')
        else:
            print(f'Request {i+1} - Ping failed')

    if successful_responses > 0:
        avg_ping = total_time / successful_responses
        packet_loss = ((num_requests - successful_responses) / num_requests) * 100
        print(f'\nAverage Ping: {avg_ping} ms')
        print(f'Packet Loss: {packet_loss}%')
    else:
        print('All requests failed.')


if __name__ == "__main__":
    dest = input('Enter a domain or IP address: ')
    num_requests = int(input('Number of requests: '))
    main(dest, num_requests)
    
# If you got warnings do these on cmd:

# 1- pip install --upgrade requests
# 2- pip install --upgrade urllib3
# 3- pip install --upgrade chardet
# 4- pip install --upgrade requests
# 5- pip install --upgrade --user urllib3==1.24.3

Enter a domain or IP address: https://www.google.com
Number of requests: 20
The IP address for www.google.com is 74.125.203.147
Request 1 - Ping: 0.40987348556518555 ms
Request 2 - Ping: 0.5882406234741211 ms
Request 3 - Ping: 0.47331714630126953 ms
Request 4 - Ping: 0.42350101470947266 ms
Request 5 - Ping: 0.4344806671142578 ms
Request 6 - Ping: 0.43054747581481934 ms
Request 7 - Ping: 0.4021155834197998 ms
Request 8 - Ping: 0.528548002243042 ms
Request 9 - Ping: 0.41410255432128906 ms
Request 10 - Ping: 0.4294090270996094 ms
Request 11 - Ping: 0.43099045753479004 ms
Request 12 - Ping: 0.43545007705688477 ms
Request 13 - Ping: 0.4098985195159912 ms
Request 14 - Ping: 0.4259350299835205 ms
Request 15 - Ping: 0.4051675796508789 ms
Request 16 - Ping: 0.42819881439208984 ms
Request 17 - Ping: 0.40590405464172363 ms
Request 18 - Ping: 0.4044039249420166 ms
Request 19 - Ping: 0.46535754203796387 ms
Request 20 - Ping: 0.4231438636779785 ms

Average Ping: 0.4384292721748352 ms
Packet Loss: 0.